# Crystal Structure on King Tableaux

Each irreducible representation of $GL_n$ has a natural basis given by semistandard tableaux, and whose character is a Schur polynomial, combinatorially thought of as a generating function over these semistandard tableaux. In $Sp_{2n}$, the irreducible representations come with a basis given by *symplectic tableaux*, proposed independently by Kashiwara/Nakashima and King. Their definitions are quite different, the former more compatible with crystal operations, and the latter more compatible with weight multiplicities and restriction to subgroups. An intricate bijection between the two tableaux was given by Sheats (https://www.ams.org/journals/tran/1999-351-09/S0002-9947-99-02166-2/S0002-9947-99-02166-2.pdf).

A good exposition on the crystal structure on KN tableaux and in other Lie types can be found in ``Crystal Bases: Representations and Combinatorics`` by Bump and Schilling.

We wish to investigate the crystal structure on King tableaux recently defined by Lee in https://arxiv.org/pdf/1910.04459.pdf and in particular see its connection to the crystal structure on KN tableaux via the Sheats bijection.

To be clear on the objects of study, the King tableaux we consider are defined as
#### Definition
A *symplectic tableaux* $T$ of shape $\lambda$ is a filling of the Ferrers diagram of $\lambda$ with the letters $1 < \overline{1} < 2 < \cdots < n < \overline{n}$ such that
- $T$ is semistandard with respect to the above ordering
- The entries $\overline{i}$ must be in row $\leq i$.


### Warning
**You might need the file** ``Symplectic Tableaux.ipynb`` **for the following code to run.**

In [6]:
%%capture
%run Symplectic\ Tableaux.ipynb

If it worked, you should be able to run the following, for example:

In [7]:
st = SymplecticTableau([[1,-1,-2],[-3]], tableau_type="Sundaram")
st.pp()

 -3
  1 -1 -2


## Generating Original Crystals

The code below is to view the crystals on KN tableaux, King tableaux, semistandard oscillating tableaux, and row reading words.

**WARNING** The crystals here are not the crystals defined in Lee, but rather generating the KN crytals in Sage and then mapping each KN tableaux to the appropriate combinatorial object. More specifically, the code to generate the aforementioned crystals is summarized as:

- KN tableaux: generate crystal from Sage
- King tableaux: generate KN crystal, then apply Sheats on each vertex
- SSOT: generate KN crystal, apply Sheats on each vertex, then apply map to SSOT on each vertex
- row reading words: SSOT crystal above, then apply map to row reading words on each vertex.

### Viewing the crystals (only run if you want pdf outputs)

In [ ]:
n = 3
p = [2]
#############
C = crystals.Tableaux(['C',n], shape=p)
C_sund = crystal_to_sundaram(C)
C_osc = crystal_to_oscillating(C)
C_rr = crystal_to_row_reading(C)
crystal_list = [C, C_sund, C_osc, C_rr]
for cl in crystal_list:
    view(cl)

### Code 

In [ ]:
from sage.graphs.graph_latex import check_tkz_graph
from sage.combinat.tableau import *

def crystal_to_oscillating(KNc):
    '''
    Return DiGraph obtained from applying Sheats bijection element-wise on
    vertices of type C crystal ``KNc`` and then bijecting to SSOT.
    '''
    n = KNc.cartan_type()[1]
    sund = crystal_to_sundaram(KNc)
    new_di = {}
    for v in sund.vertices():
        width = v.conjugate().height()
        in_vert = tuple(to_SSOT(v, width=width, height=n, is_vert=False))
        new_di[in_vert] = {}
        for outgoing in sund.outgoing_edges(v):
            out_vert = tuple(to_SSOT(outgoing[1], width=width, height=n, is_vert=False))
            new_di[in_vert][out_vert] = outgoing[2]
    D = DiGraph(new_di)
    D.set_latex_options(format='dot2tex', edge_labels=True, color_by_label=True)
    return D

def crystal_to_row_reading(KNc):
    '''
    Return DiGraph obtained from applying Sheats bijection element-wise on
    vertices of type C crystal ``KNc``, applying bijection to SSOT, then
    mapping to row reading word.
    '''
    n = KNc.cartan_type()[1]
    osc = crystal_to_oscillating(KNc)
    new_di = {}
    for v in osc.vertices():
        in_vert = to_row_reading(v)
        new_di[in_vert] = {}
        for outgoing in osc.outgoing_edges(v):
            out_vert = to_row_reading(outgoing[1])
            new_di[in_vert][out_vert] = outgoing[2]
    D = DiGraph(new_di)
    D.set_latex_options(format='dot2tex', edge_labels=True, color_by_label=True)
    return D

def crystal_to_sundaram(KNc):
    '''
    Return DiGraph obtained from applying Sheats bijection element-wise on
    vertices of type C crystal ``KNc``.
    
    Input:
    
    -``KNc`` -- a crystal of type C
    '''
    n = KNc.cartan_type()[1]
    KN_di = KNc.digraph()                   
    sund_di = {}
    for kn in KN_di.vertices():
        sund_tab = SymplecticTableau(kn.to_tableau(),max_entry=n).to_sundaram()
        sund_di[sund_tab] = {} 
        for outgoing in KN_di.outgoing_edges(kn):
            out_sund_tab = SymplecticTableau(outgoing[1].to_tableau(),max_entry=n).to_sundaram()
            sund_di[sund_tab][out_sund_tab] = outgoing[2] #outgoing[2] is the type of edge
    D = DiGraph(sund_di)
    D.set_latex_options(format='dot2tex', edge_labels=True, color_by_label=True)
    return D
    
def to_row_reading(got):
    '''
    Return row reading word of horizontal SSOT ``got``
    
    Input:
    
    - ``got`` -- a list of partitions, starting from emptyset,
                 going up-down for each step
                 
    Output:
    
    - a tuple of tuples, the i^th tuple being the rows in which
      boxes at the i^th step were added and deleted
    '''
    steps = len(got)/2
    rr = [[] for i in range(steps)]
    for s in range(steps):
        prev = got[2*s]
        up = got[2*s+1]
        down = got[2*s+2]
        
        row_up = []
        row_down = []
        #
        row_diff_up = SkewPartition([up, prev]).row_lengths()
        for r in range(len(row_diff_up)):
            if row_diff_up[r] != 0:
                row_up += [r+1 for i in range(row_diff_up[r])]
        #
        row_diff_down = SkewPartition([up, down]).row_lengths()
        for r in range(len(row_diff_down)):
            if row_diff_down[r] != 0:
                row_down += [-(r+1)]*row_diff_down[r]
        
        rr[s] = tuple(sorted(row_up + row_down)[::-1])
        
    return tuple(rr)

def to_SSOT(st, **kwargs):
    '''
    Return a Semistandard Oscillating Tableau from a Sundaram symplectic tableau.
        
    INPUT:
    
    -``st`` -- A Sundaram symplectic tableau
    
    Keyword Arguments:
    
    -``width`` -- An integer that is the width of the box that the complement will 
                  be taken in. This defaults to the size of the first row of ``st``
    
    -``height`` -- An integer that is the height of the box that the complement will
                   be taken in. This defaults to the max_entry of the tableau ``st``
    
    -``is_vert`` -- A boolean. If True, applies the bijection to vertical SSOT.
                    If False, applies the bijection to horizontal SSOT. 
                    Defaults to True.
    
    OUTPUT:
    
    - A vertical or horizontal Semistandard Oscillating Tableau.
    
    TEST::
    
        sage: st = SymplecticTableau([], max_entry=2, tableau_type="Sundaram")
        sage: to_SSOT(st)
        [[], [], [], [], []]
        sage: to_SSOT(st, width=3)
        [[], [1, 1, 1], [1, 1, 1], [2, 2, 2], [2, 2, 2]]
    '''
    if st:
        width = kwargs.get('width', len(st[0]))
    else:
        width = kwargs.get('width', 0)
    height = kwargs.get('height', st.max_entry)
    is_vert = kwargs.get('is_vert', True)
    
    # convert st to tableau with set-valued entries
    # svt is dictionary with keys the entries 1,-1,..., n,-n
    # and values the rows in which 'key' appears.
    svt = {}
    st_tr = st.conjugate().to_list() + [[] for i in range(width-len(st.conjugate()))] 
    for i in range(1,height+1):
        svt[i] = sorted([j for j in range(width) if i not in st_tr[width-1-j]])
        svt[-i] = sorted([j for j in range(width) if -i in st_tr[width-1-j]])
    
    # convert tableau with set-valued entries to sequence of tableaux
    ssot = [[0 for i in range(width)]]
    for i in range(1,height+1):
        prev = copy(ssot[-1])   
        # add box for each row in svt[i]
        for col in svt[i]:
            prev[col] += 1
        ssot += [prev]
        prev_neg = copy(prev)
        # remove box for each row in svt[-i]
        for col in svt[-i]:
            prev_neg[col] -= 1
        ssot += [prev_neg]
    
    # ssot currently a vSSOT, stored as list of lists
    if is_vert:
        ssot = [Partition(t) for t in ssot]
    else:
        ssot = [Partition(t).conjugate() for t in ssot]
    return ssot


## Generating New Crystals

### The code here is to apply the crystal operators defined by Lee. 

In [76]:
debug = False

def f(rr, i, width):
    '''
    Return row reading word of f_i crystal operator on row reading word ``rr``.
    
    Input:
    
    -``rr`` -- a tuple of tuples of integers that is the row reading word of a hSSOT
    -``i`` -- an integer from 1 to length of ``rr``.
    -``width`` -- a non-negative integer that is the maximum number of columns of
                  correspoding hSSOT (so maximum number of positive entries in each tuple in ``rr``).
    '''
    
    if i < len(rr):
        # C, D are multisets of integers 1,-1,...,n,-n.
        # They will be stored as dictionaries with keys=1,-1,...,n,-n 
        # and values being the multiplicity in the multiset.
        
        # define operations needed for C, D
        set_minus = lambda A, B: {key: max(value-B.get(key,0), 0) for (key, value) in A.items()}
        disjoint_union = lambda A, B: {key: A.get(key,0) + B.get(key,0) for key in A.keys()+B.keys()}
        negate = lambda A: {-key: value for (key, value) in A.items()}
        add = lambda A, n: {key+n: value for (key, value) in A.items()}
        intersect = lambda A, B: {key: min(A.get(key, 0), B.get(key, 0)) for key in A.keys()+B.keys()}
        up_arrow = lambda A, B: disjoint_union(set_minus(A,B), add(intersect(A,B),1))
        down_arrow = lambda A, B: disjoint_union(set_minus(A,B), add(intersect(A,B),-1))
        
        # get multisets r_plus, r_minus, r_plus_bar, etc.
        to_dict = lambda a: {j: a.count(j) for j in a}
        r_plus_i = to_dict([x for x in rr[i-1] if x > 0])
        r_minus_i = to_dict([-x for x in rr[i-1] if x < 0])
        r_plus_i_next = to_dict([x for x in rr[i] if x > 0])
        r_minus_i_next = to_dict([-x for x in rr[i] if x < 0])
        r_minus_bar_i = up_arrow(r_minus_i, r_plus_i_next)
        r_plus_bar_i_next = up_arrow(r_plus_i_next, r_minus_i)
        
        # define C, D
        C = disjoint_union(r_plus_i, negate(r_minus_bar_i))
        D = disjoint_union(r_plus_bar_i_next, negate(r_minus_i_next))
        
        # convert C, D to sorted lists for later
        to_list = lambda A: sorted([key for (key, value) in A.items() for i in range(value)])[::-1]
        srt_key = lambda k: 2*k-1 if k>0 else -2*k
        C_list = to_list(C)
        C_list_copy = deepcopy(C_list)
        D_list = to_list(D)       
        # TESTING
        if debug:
            print "rr: ", rr
            print "r_plus_i: ", r_plus_i
            print "r_minus_i: ", r_minus_i
            print "r_plus_i_next: ", r_plus_i_next
            print "r_minus_i_next: ", r_minus_i_next
            print "r_minus_bar_i: ", r_minus_bar_i
            print "r_plus_bar_i_next: ", r_plus_bar_i_next
            print "C: ", C
            print "D: ", D
            print "C_list: ", C_list
            print "D_list: ", D_list
        
        # pair off p in C and q in D if p < q. Get index of smallest unpaired p in C.
        unpair_index = -1
        iters = 0
        while min(len(C_list), len(D_list)) > 0:
            if C_list[0] < D_list[0]:
                C_list.pop(0)
                D_list.pop(0)
            else:
                C_list.pop(0)
                unpair_index = iters
            iters += 1
        if C_list != []:
            unpair_index = len(C_list_copy)-1
        
        # move smalled unpaired p in C to D.
        if unpair_index == -1:  # there was no unpaired p in C, so f_i should kill rr
            new_rr = []
        else: # move p = C[unpair_index] to D
            p = C_list_copy[unpair_index]
            C[p] -= 1
            D[p] = D.get(p, 0) + 1
            # reverse steps to get rr from C, D
            new_r_plus_i = {key: value for (key, value) in C.items() if key > 0}
            new_r_minus_bar_i = {-key: value for (key, value) in C.items() if key < 0}
            new_r_plus_bar_i_next = {key: value for (key, value) in D.items() if key > 0}
            new_r_minus_i_next = {-key: value for (key, value) in D.items() if key < 0}
            #
            new_r_minus_i = down_arrow(new_r_minus_bar_i, new_r_plus_bar_i_next)
            new_r_plus_i_next = down_arrow(new_r_plus_bar_i_next, new_r_minus_bar_i)
            new_r_i = disjoint_union(new_r_plus_i, negate(new_r_minus_i))
            new_r_i_next = disjoint_union(new_r_plus_i_next, negate(new_r_minus_i_next))
            #
            new_rr = list(rr)
            new_rr[i-1] = tuple(to_list(new_r_i))
            new_rr[i] = tuple(to_list(new_r_i_next))
    
            # MORE TESTING
            if debug:
                print "unpair_index: ", unpair_index
                print "p: ", p
                print "new C: ", C
                print "new D: ", D
                print "new_r_plus_i: ", new_r_plus_i
                print "new_r_minus_bar_i: ", new_r_minus_bar_i
                print "new_r_plus_bar_i_next: ", new_r_plus_bar_i_next
                print "new_r_minus_i_next: ", new_r_minus_i_next

    # in this else, f_i is the last crystal operator of type C
    # add {1, -1} to 1st part of ``rr`` if possible
    else:
        r_pos = [x for x in rr[0] if x > 0]
        r_neg = [x for x in rr[0] if x < 0]
        if len(r_pos) < width:
            new_r0 = r_pos + [1] + r_neg + [-1]
            new_rr = [tuple(new_r0)] + list(rr)[1:]
        else:
            new_rr = []
    
    
    return tuple(new_rr)


### Code to check that the following operations commute

$$ f_{n-i} \circ \text{row reading} \circ \text{SSOT} \circ \text{Sheats} \circ KN = \text{row reading} \circ \text{SSOT} \circ \text{Sheats} \circ f_i \circ KN $$

In [ ]:
n=3
shape = [3,1,1]
width = shape[0]
####################
KNcrystal = crystals.Tableaux(['C',n], shape=shape)
for v in KNcrystal:
    sund = SymplecticTableau(v.to_tableau(),max_entry=n).to_sundaram()
    hSSOT = to_SSOT(sund, width=width, height=n, is_vert=False)
    rr = to_row_reading(hSSOT)
    for i in range(1,n+1):
        v_f = v.f(i)
        # change to row reading word
        if v_f is None:
            rr_of_vf = []
        else:
            sund_f = SymplecticTableau(v_f.to_tableau(),max_entry=n).to_sundaram()
            hSSOT_f = to_SSOT(sund_f, width=width, height=n, is_vert=False)
            rr_of_vf = to_row_reading(hSSOT_f)
        # apply f_i crystal operator on row reading word of v
        # and check it equals row reading word of f_i(v)
        if i < n:
            f_of_rr = f(rr, n-i, width)
        else:
            f_of_rr = f(rr, i, width)
        if f_of_rr != tuple(rr_of_vf):
            print "False"
            print "------------------------------------------------"
            print "before : ", v, sund, hSSOT, rr, n-i, f_of_rr
            print "after: ", v_f, sund_f, hSSOT_f, i, rr_of_vf
print "All true"

## Miscellaneous Code

In [ ]:
def from_row_reading(rr):
    '''
    Return hSSOT from row reading word ``rr``.    
    '''
    steps = len(rr)
    hSSOT = [Partition([])]
    for s in range(steps):
        up = hSSOT[2*s]
        for row in rr[s]:
            if row > 0:
                up = up.add_cell(row-1)
        down = up
        for row in rr[s]:
            if row < 0:
                down = down.remove_cell(-row-1)
        hSSOT += [up, down]
    
    return hSSOT

## Sandbox